In [1]:
import os
import sys
import time
import warnings
warnings.filterwarnings('ignore')
# libraries for visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(style="white")
import pandas as pd
pd.set_option("display.max_columns", 30)
import numpy as np
from datetime import datetime
# Using make_column_transformer to One-Hot Encode
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from prince import mca

In [2]:
directory_path = os.path.abspath(os.path.join('..'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
os.chdir(directory_path)
os.getcwd()

'/Users/iairlinker/Documents/repos/flight_delays_challenge'

In [3]:
start_notebook = time.time()

In [4]:
def get_na(df):
    qsna=df.shape[0]-df.isnull().sum(axis=0)
    qna=df.isnull().sum(axis=0)
    ppna=round(100*(df.isnull().sum(axis=0)/df.shape[0]),2)
    aux= {'datos sin NAs en q': qsna, 'Na en q': qna ,'Na en %': ppna}
    na=pd.DataFrame(data=aux)
    return na.sort_values(by='Na en %',ascending=False)

In [ ]:
# Loading the joined dataset between flights and Airport database
path = os.getcwd() + '/data/processed/database.csv'
df = pd.read_csv(path, sep=',')

# create the variable of number of flights in destiny by date and merge it with the database
aux = df.groupby(['fecha_operacion'])['Ori-O'].count().reset_index().sort_values(by='Ori-O')
aux.rename(columns={'Ori-O':'NVuelos-Ori-O'}, inplace=True)
df = pd.merge(df, aux, on = ['fecha_operacion'], how='left')

# create the variable of number of flights in operated destiny by date of operation and airline who operated it
aux = df.groupby(['fecha_operacion','Emp-O','Des-O'])['Ori-O'].count().reset_index().sort_values(by='Ori-O')
aux.rename(columns={'Ori-O':'Nvuelos-Fecha-Emp-Des-O'}, inplace=True)
df = pd.merge(df, aux, on = ['fecha_operacion','Emp-O','Des-O'], how='left')

# create the variable of number of flights in destiny scheduled by date scheduled and airline scheduled
aux = df.groupby(['fecha_operacion','Emp-I','Des-I'])['Ori-I'].count().reset_index().sort_values(by='Ori-O')
aux.rename(columns={'Ori-I':'Nvuelos-Fecha-Emp-Des-I'}, inplace=True)
df = pd.merge(df, aux, on = ['fecha_operacion','Emp-I','Des-I'], how='left')




df.drop(columns=['Fecha-O','Fecha-I','AÑO','Des-I','Vlo-I','Vlo-O','Ori-O','Ori-I','IATA Code','Airport Name',
                 'Country','City/Town','ICAO Code','SIGLADES','SIGLAORI','Lat','Long','distance_to_destiny',
                 'OPERA','Emp-I','dif_min','temporada_alta'],inplace=True)
print(df.shape)
df.head(2)

In [ ]:
get_na(df)

In [ ]:
transformer = make_column_transformer(
    (OneHotEncoder(drop='first',sparse=False), ['Des-O','Emp-O','TIPOVUELO','periodo_dia','DIANOM','MES','DIA']),
    remainder='passthrough')

transformed = transformer.fit_transform(df)
transformed_df = pd.DataFrame(
    transformed, 
    columns=transformer.get_feature_names()
)
transformed_df.head()

In [ ]:
path = os.getcwd() + '/data/interim/model_data.csv' 
transformed_df.to_csv(path,
          sep=',',
          index = False
         )

## MCA

I will perform

In [ ]:
# Loading the joined dataset between flights and Airport database
path = os.getcwd() + '/data/processed/database.csv'
df = pd.read_csv(path, sep=',')
df.drop(columns=['Fecha-O','Fecha-I','AÑO','Des-I','Vlo-I','Vlo-O','Ori-O','Ori-I','IATA Code','Airport Name',
                 'Country','City/Town','ICAO Code','SIGLADES','SIGLAORI','Lat','Long','distance_to_destiny',
                 'OPERA','Emp-I','dif_min','temporada_alta','atraso_15'],inplace=True)
print(df.shape)
df.head(2)

In [ ]:
results = []
n = []
for n_comp in range(0,100):
    model = mca.MCA(
        n_components=n_comp,
        n_iter=100,
        copy=True,
        check_input=True,
        engine='auto',
        random_state=42
    )
    mca_data = model.fit(df)
    mca_x = mca_data.transform(df)
    results.append(model.total_inertia_)
    n.append(n_comp)

In [ ]:
plt.plot(n,results);

In [ ]:
model = mca.MCA(
    n_components=20,
    n_iter=100,
    copy=True,
    check_input=True,
    engine='auto',
    random_state=42
)

In [ ]:
mca_data = model.fit(df)
mca_x = mca_data.transform(df)

In [ ]:
path = os.getcwd() + '/data/interim/model_data_mca.csv' 
mca_x.to_csv(path,
          sep=',',
          index = False
         )